# Feed-forward сети

Итак, давайте потренируемся тренировать нейронные сети прямого распространения (так как делали на паре)
При этом попробуем создать свою функцию активации на одном из слоев

In [4]:
# Сделаем необходимые импорты
import torch
from torch import nn
import numpy as np
import torchvision
import torchvision.transforms as transforms

In [9]:
# Загрузим датасет CIFAR-100, сразу же создадим dataloader для него
# Если вам не хватает вычислительных ресурсов, то можно вернуться к CIFAR-10
train_dataset = torchvision.datasets.CIFAR10(root='data',
                                             train=True,  
                                             transform=transforms.ToTensor(), 
                                             download=True)



image, label = train_dataset[0]
print (image.size())
print (label)


Extracting data/cifar-10-python.tar.gz to data
torch.Size([3, 32, 32])
6


In [11]:
# Создайте собственную архитектуру! Можете использовать все, что угодно, но в ограничении на использование линейные слои (пока без сверток)

class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        #Замечу что для классификации изображений обычно испольщуют схему пирамиды для количества нейронов в скрытых слоях
        self.fc1 = nn.Linear(input_dim, 4 * hidden_dim)
        self.fc2 = nn.Linear(4 * hidden_dim, 2 * hidden_dim)
        self.fc3 = nn.Linear(2 * hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        #x.view чтоб смотреть структуру сети
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = nn.functional.leaky_relu(x, 0.05)
        x = self.fc2(x)
        x = nn.functional.leaky_relu(x, 0.05)
        x = self.fc3(x)
        x = nn.functional.leaky_relu(x, 0.05)
        x = self.fc4(x)
        return x
    
    def predict(self, x):
        x = x.view(x.shape[0], -1)
        
        x = self.fc1(x)
        x = F.leaky_relu(x, 0.05)
        x = self.fc2(x)
        x = F.leaky_relu(x, 0.05)
        x = self.fc3(x)
        x = F.leaky_relu(x, 0.05)
        x = self.fc4(x)
        x = F.softmax(x)
        return x
# Давайте добавим ограниченный Leaky_relu, то есть output = max(0.1x, 0.5x)
# Ваша задача добавить его в архитектуру сети как функцию активации

In [12]:
#размер картинки 3*32*32 = 3072
#создаем экхемпляр сеть
net = Net(3072, 100, 10)

In [16]:
from torch import optim

In [17]:
criterion = nn.CrossEntropyLoss()
#для коассификации подходит функция ошибки crossentropy 

#стохастический градиентный спуск. направдение анти-градиента определяется только по одному параметру что увеличивает скорость алгоритма
#но уменьшает скорость сходимости. lr=0.001 является тривиальным значением для шага градиента.
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [4]:
# Запустить обучение (по аналогии с тем, что делали на паре)

In [19]:
from tqdm import tqdm
#progress bar удобно

In [22]:
#будес обучаться batch-ами по 64 примера
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

In [23]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]

[1,     1] loss: 0.001
[1,   301] loss: 0.344
[1,   601] loss: 0.337


 10%|█         | 1/10 [00:14<02:09, 14.42s/it]

[2,     1] loss: 0.001
[2,   301] loss: 0.310
[2,   601] loss: 0.302


 20%|██        | 2/10 [00:26<01:50, 13.80s/it]

...

[9,     1] loss: 0.001
[9,   301] loss: 0.244
[9,   601] loss: 0.242


 90%|█████████ | 9/10 [02:34<00:16, 16.96s/it]

[10,     1] loss: 0.001
[10,   301] loss: 0.242
[10,   601] loss: 0.236


100%|██████████| 10/10 [02:50<00:00, 17.08s/it]

Training is finished!
